In [3]:
import os
from dotenv import load_dotenv

# --- Load environment variables ---
load_dotenv()

# Fetch keys and URLs from your .env file
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
ADK_PROXY_URL = os.getenv("ADK_PROXY_URL")

# Safety check
if not GOOGLE_API_KEY:
    raise ValueError("❌ GOOGLE_API_KEY not found. Please check your .env file.")
if not ADK_PROXY_URL:
    raise ValueError("❌ ADK_PROXY_URL not found. Please check your .env file.")

# Set them as environment variables for ADK to pick up
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["ADK_PROXY_URL"] = ADK_PROXY_URL

print("✅ Environment variables loaded successfully!")
print(f"🔗 Using ADK Proxy URL: {ADK_PROXY_URL}")


✅ Environment variables loaded successfully!
🔗 Using ADK Proxy URL: http://127.0.0.1:8000


In [5]:
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [6]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

###  Helper functions

We'll define some helper functions. If you are running this outside the Kaggle environment, you don't need to do this.

In [14]:
# Define helper functions that will be reused throughout the notebook

from IPython.display import display, HTML
# The ADK web UI will usually be available at http://127.0.0.1:8000 directly.
def get_adk_proxy_url_only():
    # This function's logic is specific to Kaggle Notebooks
    from jupyter_server.serverapp import list_running_servers

    PROXY_HOST = "https://kkb-production.jupyter-proxy.kaggle.net"
    ADK_PORT = "8000"

    servers = list(list_running_servers())
    if not servers:
        raise Exception("No running Jupyter servers found.")

    baseURL = servers[0]["base_url"]

    try:
        path_parts = baseURL.split("/")
        kernel = path_parts[2]
        token = path_parts[3]
    except IndexError:
        raise Exception(f"Could not parse kernel/token from base URL: {baseURL}")

    url_prefix = f"/k/{kernel}/{token}/proxy/proxy/{ADK_PORT}"
    url = f"{PROXY_HOST}{url_prefix}"

    styled_html = f"""
    <div style="padding: 15px; border: 2px solid #f0ad4e; border-radius: 8px; background-color: #fef9f0; margin: 20px 0;">
        <div style="font-family: sans-serif; margin-bottom: 12px; color: #333; font-size: 1.1em;">
            <strong>⚠️ IMPORTANT: Action Required</strong>
        </div>
        <div style="font-family: sans-serif; margin-bottom: 15px; color: #333; line-height: 1.5;">
            The ADK web UI is <strong>not running yet</strong>. You must start it in the next cell.
            <ol style="margin-top: 10px; padding-left: 20px;">
                <li style="margin-bottom: 5px;"><strong>Run the next cell</strong> (the one with <code>!adk web ...</code>) to start the ADK web UI.</li>
                <li style="margin-bottom: 5px;">Wait for that cell to show it is "Running" (it will not "complete").</li>
                <li>Once it's running, <strong>return to this button</strong> and click it to open the UI.</li>
            </ol>
            <em style="font-size: 0.9em; color: #555;">(If you click the button before running the next cell, you will get a 500 error.)</em>
        </div>
        <a href='{url}' target='_blank' style="
            display: inline-block; background-color: #1a73e8; color: white; padding: 10px 20px;
            text-decoration: none; border-radius: 25px; font-family: sans-serif; font-weight: 500;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2); transition: all 0.2s ease;">
            Open ADK Web UI (after running cell below) ↗
        </a>
    </div>
    """

    display(HTML(styled_html))

    return url_prefix

print("✅ Helper functions defined.")


✅ Helper functions defined.


### Define your agent

Now, let's build our agent. We'll configure an `Agent` by setting its key properties, which tell it what to do and how to operate.


These are the main properties we'll set:

- **name** and **description**: A simple name and description to identify our agent.
- **model**: The specific LLM that will power the agent's reasoning. We'll use "gemini-2.5-flash-lite".
- **instruction**: The agent's guiding prompt. This tells the agent what its goal is and how to behave.
- **tools**: A list of tools that the agent can use. To start, we'll give it the `google_search` tool, which lets it find up-to-date information online.

In [8]:
root_agent = Agent(
    name="Architechture_assistant",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="A simple agent that can answer questions regarding architecture.",
    instruction="You are a helpful assistant. Use Google Search for current info or if unsure.",
    tools=[google_search],
)

print("✅ Root Agent defined.")

✅ Root Agent defined.


### Run your agent

Now it's time to bring your agent to life and send it a query. To do this, you need a [`Runner`], which is the central component within ADK that acts as the orchestrator. It manages the conversation, sends our messages to the agent, and handles its responses.

**a. Create an `InMemoryRunner` and tell it to use our `root_agent`:**

In [9]:
runner = InMemoryRunner(agent=root_agent)

print("✅ Runner created.")

App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "C:\Users\surya\Downloads\.venv\Lib\site-packages\google\adk\agents", which implies app name "agents".


✅ Runner created.


👉 Note that we are using the Python Runner directly in this notebook. You can also run agents using ADK command-line tools such as `adk run`, `adk web`, or `adk api_server`. To learn more, check out the documentation related to [runtime in ADK].

**b. Now you can call the `.run_debug()` method to send our prompt and get an answer.**

This method abstracts the process of session creation and maintenance and is used in prototyping. We'll explore "what sessions are and how to create them" on Day 3.

In [10]:
response = await runner.run_debug(
    "How do you stay current with industry trends and advancements in architecture? "
)


 ### Created new session: debug_session_id

User > How do you stay current with industry trends and advancements in architecture? 
Architechture_assistant > As an AI, I don't "stay current" in the same way a human professional does. I don't attend conferences, read physical journals, or network with peers. Instead, my knowledge base is constantly updated through the vast amounts of data I am trained on.

Here's how that process allows me to be aware of industry trends and advancements in architecture:

*   **Continuous Data Ingestion:** My training data includes a massive and ever-growing collection of text and code. This encompasses a wide range of sources, including:
    *   **Architectural Publications:** Digital archives of leading architectural magazines, journals, and online platforms.
    *   **Academic Research:** Papers, studies, and dissertations from architectural schools and research institutions.
    *   **Industry News and Blogs:** Articles, reports, and analyses from re

You can see a summary of ADK and its available languages in the response.

###  How does it work?

The agent performed a Google Search to get the latest information about ADK, and it knew to use this tool because:

1. The agent inspects and is aware of which tools it has available to use.
2. The agent's instructions specify the use of the search tool to get current information or if it is unsure of an answer.

The best way to see the full, detailed trace of the agent's thoughts and actions is in the **ADK web UI**, which we'll set up later in this notebook.

And we'll cover more detailed workflows for logging and observability later in the course.

In [11]:
response = await runner.run_debug("How would you describe your design philosophy?")


 ### Continue session: debug_session_id

User > How would you describe your design philosophy?
Architechture_assistant > As an AI, I don't have personal experiences, emotions, or a subjective "design philosophy" in the human sense. I don't create designs out of personal inspiration or adhere to a particular aesthetic.

However, if I were to describe the *principles that guide my responses and information delivery* regarding architectural design, they would be:

*   **Informativeness and Accuracy:** My primary goal is to provide comprehensive, accurate, and up-to-date information about architectural concepts, historical styles, contemporary trends, building technologies, and relevant theories.
*   **Objectivity and Neutrality:** I strive to present information without bias, acknowledging diverse perspectives and approaches within the field of architecture. I aim to describe rather than advocate for a specific style or method.
*   **Clarity and Accessibility:** I aim to explain complex 

In [15]:
!adk create sample-agent --model gemini-2.5-flash-lite --api_key $GOOGLE_API_KEY 

^C


In [ ]:
!adk web

^C


C:\Users\surya\Downloads\.venv\Lib\site-packages\google\adk\cli\fast_api.py:130: UserWarning: [EXPERIMENTAL] InMemoryCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.

+-----------------------------------------------------------------------------+

  credential_service = InMemoryCredentialService()

| ADK Web Server started                                                      |

...

| For local testing, access at http://127.0.0.1:8000.                         |

INFO:     Started server process [21060]

+-----------------------------------------------------------------------------+

INFO:     Waiting for application startup.


INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
